In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score,f1_score

In [2]:
# Load Data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

In [3]:
pip install "xlrd>=2.0.1"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
df = pd.read_excel(url)

In [5]:
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [6]:
# Skip first header row as the dataset has double headers
df = pd.read_excel(url, header=1)

In [7]:
# Rename target column
df.rename(columns={'default payment next month': 'target'}, inplace=True)

In [8]:
X = df.drop(['ID', 'target'], axis=1) # Drop ID and Target
y = df['target']

In [10]:
y.value_counts()

target
0    23364
1     6636
Name: count, dtype: int64

In [11]:
# stratified shuffling for imbalanced class 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Scaling (CRITICAL for ANN Accuracy)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Bulding the hypermodel

In [13]:
def build_model(hp):
    model = Sequential()
    
    # input_dim matches the number of features (23)
    input_dim = X_train_scaled.shape[1]
    
    # Tune the number of hidden layers (between 1 and 4)
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Dense(
            # Tune number of units separately for each layer
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation', ['relu', 'swish']), 
        ))
        
        # Tune Batch Normalization (helps convergence)
        if hp.Boolean('batch_norm'):
            model.add(BatchNormalization())
            
        # Tune Dropout rate to prevent overfitting
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))

    # Output Layer (Binary Classification)
    model.add(Dense(1, activation='sigmoid'))
    
    # Tune Learning Rate
    learning_rate = hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='log')
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [17]:
# Initialize Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,             # Total distinct models to test
    executions_per_trial=1,    # Train each model once 
    directory='my_dir',
    project_name='credit_card_churn_random' 
)

In [18]:
# Callbacks for the search phase
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [19]:
print("Starting Hyperparameter Search...")
tuner.search(X_train_scaled, y_train, epochs=20, validation_split=0.2, callbacks=[stop_early])

Trial 20 Complete [00h 00m 12s]
val_accuracy: 0.8147916793823242

Best val_accuracy So Far: 0.8172916769981384
Total elapsed time: 00h 04m 44s


In [20]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [22]:
best_hps.values

{'num_layers': 1,
 'units_0': 64,
 'activation': 'swish',
 'batch_norm': False,
 'dropout': 0.0,
 'lr': 0.00293879455768389,
 'units_1': 32,
 'units_2': 288,
 'units_3': 256}

## Training the best Model

In [39]:
# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

In [40]:
# Advanced Callbacks for final training
callbacks = [
    # Stop if validation loss doesn't improve for 10 epochs (restore best weights)
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    # If loss plateaus, reduce learning rate by 20% to fine-tune
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

In [29]:
# Train the best model
history = best_model.fit(
    X_train_scaled, 
    y_train, 
    epochs=200,             # Higher epochs, EarlyStopping will handle the rest
    validation_split=0.2, 
    batch_size=32, 
    callbacks=callbacks
)

Epoch 1/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8083 - loss: 0.4696 - val_accuracy: 0.8062 - val_loss: 0.4649 - learning_rate: 0.0029
Epoch 2/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8176 - loss: 0.4445 - val_accuracy: 0.7969 - val_loss: 0.4650 - learning_rate: 0.0029
Epoch 3/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8187 - loss: 0.4384 - val_accuracy: 0.8083 - val_loss: 0.4646 - learning_rate: 0.0029
Epoch 4/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8205 - loss: 0.4363 - val_accuracy: 0.8169 - val_loss: 0.4543 - learning_rate: 0.0029
Epoch 5/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8213 - loss: 0.4345 - val_accuracy: 0.8133 - val_loss: 0.4588 - learning_rate: 0.0029
Epoch 6/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8196 - loss: 0.4318 - val_accuracy: 0.8140 - val_loss: 0.4522 - learning_rate: 0.0029
Epoch 7/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8220 - loss: 0.

- Model stopped early


## Final evaluation

In [35]:
# Predict and Evaluate
y_pred_prob = best_model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\n--- Final F1_score ---")
print(f"Accuracy: {f1_score(y_test, y_pred):.4f}")
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step

--- Final F1_score ---
Accuracy: 0.4845

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      4673
           1       0.64      0.39      0.48      1327

    accuracy                           0.82      6000
   macro avg       0.74      0.66      0.69      6000
weighted avg       0.80      0.82      0.80      6000



## Improving the Recall

- The given problem is of unbalanced classification due to which despite having higher accuracy the recall is 39%.
-  we can calculate and assign weights using sklearn to tackel this situation

In [36]:
from sklearn.utils import class_weight

In [37]:
# Calculate weights based on training data
weights = class_weight.compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)

In [38]:
weights_dict = dict(enumerate(weights))

- Now training the model again with weights
- note: to keep track see the cell runing indices

In [46]:
history = best_model.fit(
    X_train_scaled, 
    y_train, 
    epochs=200,                  # Higher epochs, EarlyStopping will handle the rest
    class_weight=weights_dict,
    validation_split=0.2, 
    batch_size=32, 
    callbacks=callbacks
)

Epoch 1/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7734 - loss: 0.5578 - val_accuracy: 0.7581 - val_loss: 0.5709 - learning_rate: 7.3470e-04
Epoch 2/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7699 - loss: 0.5560 - val_accuracy: 0.7658 - val_loss: 0.5653 - learning_rate: 7.3470e-04
Epoch 3/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7706 - loss: 0.5557 - val_accuracy: 0.7598 - val_loss: 0.5720 - learning_rate: 7.3470e-04
Epoch 4/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7709 - loss: 0.5547 - val_accuracy: 0.7604 - val_loss: 0.5663 - learning_rate: 7.3470e-04
Epoch 5/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7658 - loss: 0.5546 - val_accuracy: 0.7681 - val_loss: 0.5557 - learning_rate: 7.3470e-04
Epoch 6/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7714 - loss: 0.5530 - val_accuracy: 0.7633 - val_loss: 0.5690 - learning_rate: 3.6735e-04
Epoch 7/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc

- Model stopped early


## Final re-evaluation

In [47]:
# Predict and Evaluate
y_pred_prob = best_model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\n--- Final F1_score ---")
print(f"Accuracy: {f1_score(y_test, y_pred):.4f}")
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step

--- Final F1_score ---
Accuracy: 0.5234

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.88      0.79      0.83      4673
           1       0.46      0.61      0.52      1327

    accuracy                           0.75      6000
   macro avg       0.67      0.70      0.68      6000
weighted avg       0.79      0.75      0.77      6000



## Results
- It can be clearly seen that we have a significant improvement in recall by 22% with a default probability of 0.5


## Suggestions
- We use optimized probability as per a given use case where probability could be lowe than 0.5 and with a even higher recall